<a href="https://colab.research.google.com/github/cdmurray80/object_detection/blob/main/object_detection1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Docs: https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/2.2.0/install.html#tensorflow-object-detection-api-installation

In [2]:
# Mount drive
#
#
from google.colab import drive
drive.mount('/content/drive', force_remount=True) 

Mounted at /content/drive


In [3]:
def do_setup():
  %pip install --ignore-installed --upgrade tensorflow
  import os
  if not os.path.isdir('/content/tensorflow'):
    %mkdir tensorflow
  %cd /content/tensorflow
  !git clone --depth 1 https://github.com/tensorflow/models

  %cd /content/tensorflow/models/research
  %ls
  !protoc object_detection/protos/*.proto --python_out=.

  %cd /content/tensorflow/models/research
  %ls
  !protoc object_detection/protos/*.proto --python_out=.

  %cd /content/tensorflow/models/research
  %ls
  !protoc object_detection/protos/*.proto --python_out=.

  %pip install cython
  %pip install git+https://github.com/philferriere/cocoapi.git#subdirectory=PythonAPI

  %cd /content/tensorflow/models/research
  !cp object_detection/packages/tf2/setup.py .
  !python -m pip install --use-feature=2020-resolver .

def test_setup():
  !python object_detection/builders/model_builder_tf2_test.py


In [4]:
do_setup()
test_setup()

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 511.7 MB 6.2 kB/s 
     |████████████████████████████████| 40 kB 7.6 MB/s 
     |████████████████████████████████| 4.1 MB 48.9 MB/s 
     |████████████████████████████████| 65 kB 5.7 MB/s 
     |████████████████████████████████| 4.4 MB 56.9 MB/s 
     |████████████████████████████████| 42 kB 1.4 MB/s 
     |████████████████████████████████| 1.6 MB 50.7 MB/s 
     |████████████████████████████████| 5.8 MB 55.6 MB/s 
     |████████████████████████████████| 438 kB 74.6 MB/s 
     |████████████████████████████████| 75 kB 4.6 MB/s 
     |████████████████████████████████| 57 kB 7.1 MB/s 
     |████████████████████████████████| 1.2 MB 49.6 MB/s 
     |████████████████████████████████| 14.5 MB 60.5 MB/s 
     |████████████████████████████████| 15.7 MB 53.9 MB/s 
     |████████████████████████████████| 123 kB 77.1 MB/s 
     |████████████████████████████████

/content/tensorflow
Cloning into 'models'...
remote: Enumerating objects: 3391, done.
remote: Counting objects: 100% (3391/3391), done.
remote: Compressing objects: 100% (2816/2816), done.
remote: Total 3391 (delta 895), reused 1380 (delta 520), pack-reused 0
Receiving objects: 100% (3391/3391), 34.93 MiB | 34.19 MiB/s, done.
Resolving deltas: 100% (895/895), done.
/content/tensorflow/models/research
adversarial_text/    deeplab/                marco/             seq_flow_lite/
attention_ocr/       deep_speech/            nst_blogpost/      slim/
audioset/            delf/                   object_detection/  vid2depth/
autoaugment/         efficient-hrl/          pcl_rl/
cognitive_planning/  lfads/                  README.md
cvt_text/            lstm_object_detection/  rebar/
/content/tensorflow/models/research
adversarial_text/    deeplab/                marco/             seq_flow_lite/
attention_ocr/       deep_speech/            nst_blogpost/      slim/
audioset/            delf/ 

In [5]:
import os
WORKSPACE_DIR='/content/tensorflow/workspace'
test_dir=WORKSPACE_DIR+'/training_demo/images/test'
train_dir=WORKSPACE_DIR+'/training_demo/images/train'
model_dir=WORKSPACE_DIR+'/training_demo/pre-trained-models'
annotations_dir=WORKSPACE_DIR + '/training_demo/annotations'
for d in [WORKSPACE_DIR, WORKSPACE_DIR+'/training_demo', WORKSPACE_DIR+'/training_demo/images', annotations_dir, WORKSPACE_DIR+'/training_demo/scripts', train_dir, test_dir, model_dir]:
  if not os.path.exists(d):
    os.mkdir(d)
    print('Created directory', d)

Created directory /content/tensorflow/workspace
Created directory /content/tensorflow/workspace/training_demo
Created directory /content/tensorflow/workspace/training_demo/images
Created directory /content/tensorflow/workspace/training_demo/annotations
Created directory /content/tensorflow/workspace/training_demo/scripts
Created directory /content/tensorflow/workspace/training_demo/images/train
Created directory /content/tensorflow/workspace/training_demo/images/test
Created directory /content/tensorflow/workspace/training_demo/pre-trained-models


In [7]:
#Get all xml files
import glob,shutil
import xml.etree.ElementTree as ET
paths=['/content/drive/MyDrive/Research/object_detection/images/final_MLRS', '/content/drive/MyDrive/Research/object_detection/images/final_t72']
xml_files=[glob.glob(path+'/*.xml') for path in paths]
xml_files=[f for l in xml_files for f in l]

#Move to train and test
for i,f in enumerate(xml_files):
  dir = test_dir if i % 5 == 4 else train_dir
  #Get name of actual image file
  tree=ET.parse(f)
  root=tree.getroot()
  img_file=root[1].text
  f2=f.replace('.xml', '.'+ img_file.split('.')[-1])
  shutil.copy2( f, dir)
  shutil.copy2( f2, dir)

In [8]:
#Create label file
labels=['MLRS', 'tank']
s=''
for i,l in enumerate(labels):
  s+='''item {\n \t id: %d\n \t name:'%s'\n}\n\n''' % (i+1, l)
#Write file
label_file=annotations_dir+'/label_map.pbtxt'
text_file = open(label_file, 'w')
n = text_file.write(s)
text_file.close()

In [9]:
#Script to create tfrecords
src = '/content/drive/MyDrive/Research/object_detection/generate_tfrecord.py'
dst = WORKSPACE_DIR+'/training_demo/scripts'
shutil.copy2(src, dst)

'/content/tensorflow/workspace/training_demo/scripts/generate_tfrecord.py'

In [10]:
# Create train and test tfrecords
train_record_file=annotations_dir+'/train.record'
test_record_file=annotations_dir+'/test.record'
!python /content/drive/MyDrive/Research/object_detection/generate_tfrecord.py -x $train_dir -l $label_file -o $train_record_file
!python /content/drive/MyDrive/Research/object_detection/generate_tfrecord.py -x $test_dir -l $label_file -o $test_record_file


Successfully created the TFRecord file: /content/tensorflow/workspace/training_demo/annotations/train.record
Successfully created the TFRecord file: /content/tensorflow/workspace/training_demo/annotations/test.record


In [11]:
import requests

# More models are here

url = 'http://download.tensorflow.org/models/object_detection/tf2/20200711/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz'
target_file = WORKSPACE_DIR + '/training_demo/pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz'
print(target_file)
response = requests.get(url)
open(target_file, "wb").write(response.content)

/content/tensorflow/workspace/training_demo/pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8.tar.gz


244817203

In [12]:
%cd /content/tensorflow/workspace/training_demo/pre-trained-models
!tar -xvzf $target_file 

/content/tensorflow/workspace/training_demo/pre-trained-models
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.data-00000-of-00001
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/checkpoint
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/checkpoint/ckpt-0.index
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/saved_model.pb
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/assets/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/variables.data-00000-of-00001
ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/saved_model/variables/variables.index


In [13]:
#One time deal
shutil.copy2('/content/drive/MyDrive/Research/object_detection/pipeline.config', '/content/tensorflow/workspace/training_demo/pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/')

'/content/tensorflow/workspace/training_demo/pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config'

In [14]:
shutil.copy2('/content/tensorflow/models/research/object_detection/model_main_tf2.py', WORKSPACE_DIR+'/training_demo')
%cd /content/tensorflow/workspace/training_demo

/content/tensorflow/workspace/training_demo


In [15]:
%pip install "opencv-python-headless<4.3"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 21.6 MB 1.5 MB/s 
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.6.0.66
    Uninstalling opencv-python-headless-4.6.0.66:
      Successfully uninstalled opencv-python-headless-4.6.0.66


In [ ]:
# Actually train!
!python model_main_tf2.py --model_dir=pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/ --pipeline_config_path=pre-trained-models/ssd_resnet50_v1_fpn_640x640_coco17_tpu-8/pipeline.config